In [70]:
import zipfile
import os
import configparser
import psycopg2

config = configparser.ConfigParser()
config.read('./python-etl/etl.cfg')
print(*config['DWH'].values())

localhost melhorenvio melhorenviouser melhorenviopassword 5432


In [151]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['DWH'].values()))
cur = conn.cursor()

In [64]:
## Specify the path to your zip file
#zip_path = 'data.zip'

## Open the zip file in read mode
#with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#    # Extract all the contents into the current directory
#    zip_ref.extractall('./data')

In [65]:
# List to store the names of CSV files
csv_files = []

In [66]:
# Loop through each file in the specified folder
for file_name in os.listdir('./data'):
    # Check if the file is a CSV by its extension
    if file_name.endswith('.csv'):
        csv_files.append(file_name)
    
# Sort the list of CSV file names
csv_files_sorted = sorted(csv_files)

In [152]:
sql_query_create = f"""
    CREATE TABLE teste1 (
      Op VARCHAR(50),
      oid__id UUID,
      createdAt BIGINT,
      updatedAt BIGINT,
      lastSyncTracker BIGINT,
      array_trackingEvents JSON,
      PRIMARY KEY (oid__id)
        );
    """
cur.execute(sql_query_create)
conn.commit()

In [153]:
import csv

with open('20230904-130010492.csv', mode='r') as file:
    sql_query = f"""
    COPY teste1 
    FROM STDIN
    WITH (FORMAT csv, HEADER true, DELIMITER ',');
    """
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['DWH'].values()))
    cur = conn.cursor()
    cur.copy_expert(sql_query, file)
    conn.commit()
    cur.close()

InvalidTextRepresentation: invalid input syntax for type json
DETAIL:  Token "'" is invalid.
CONTEXT:  JSON data, line 1: [{'...
COPY teste1, line 2, column array_trackingevents: "[{'createdAt': {'$date': 1690994210000}, 'trackingCode': '2969a474d802523cf034c17c5f4ef5e2', 'status..."


QueryCanceled: COPY from stdin failed: error in .read() call: ValueError I/O operation on closed file.
CONTEXT:  COPY teste1, line 1


In [91]:
# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()